<h1>Surprise - 파이썬 추천 패키지</h1>


<img src="./img/SuprisePackage1.png">

<h2>Surprise 패키지를 이용한 추천 수행 프로세스</h2>

데이터를 Dataset이라는 객체로 로딩을 한다. 그리고 모델 설정 및 학습을 하여 행렬 기반분해(SVD)를 할거냐 아이템 기반의 협업 필터를 할거냐

(KNNBasic)를 설정한다. 그 다음에 학습을 한다(train()메소드) 그 다음에 예측 및 평가를 한다.

약간 사이킷런이랑 다른게 test()와 predict()가 있는데 사이킷런의 predict()이랑 같은건 test()이다. 

입력 인자에서 들어온 모든 dataset에 대한 예측 평가를 해주는데 predict()는 한건만 해준다.

그리고 cross_validate를 할 수 있고 GridSerchCV로 하이퍼 파라미터 튜닝과 cross_validation을 함께 할 수 있다.

조금 유의할 점은 데이터 로딩 부분인데, Surprise 패키지는  '반드시 이런 데이터 유형으로 들어와야 됩니다'라고 고정이 되어있는게 있는데

UserId, ItemId, rating이 Raw 레벨로 되어있어야 한다. 

<img src = "./img/SurpriseProcess.png">

<h2>Surprise를 이용한 추천 구현 기본</h2>

순서는 이렇다

앞에랑의 차이가, 사이킷런은 predict였는데 surprise는 test()이다. 결과가 리스트 형으로 나온다.


<img src = "./img/SurpriseRecom1.png">

<h2>Surprise 주요 모듈 소개 - Dataset</h2>

Surprise는 기본적으로 Fix된 형태를 가져간다고 했다.(low 레벨로)

우리가 지금까지 학습을 했던거는 pivot_table을 해서 바꿨었다.

하지만 입력 자체는 이렇게 Raw 레벨로 되어있어야 한다. Raw 레벨로 데이터가 들어가게 되면 데이터셋이 알아서 변환을 해준다.

그리고 첫번째 두번째 세번째 칼럼을  무조건 사용자 I 아이템 ID Rating으로 하고 4번째부터는 아예 관여를 하지 않는다.

<img src = "./img/SurpriseDataset.png">

<h2>Dataset 클래스의 주요 메소드</h2>

대표적으로 load_builtin 이게 있다. 알아서 다 맞춰준다. 

무비렌즈 데이터 같은 내장 데이터 말고 일반적으로 OS 파일이나 데이터프레임에서 로딩을 한다.

load_from_file을 쓰면 file_path:파일이 어디 있는지 두번째는 reader: 이 클래스를 이용하면 포맷팅을 해준다. 뒤에서 설명함.

load_from_df도 마찬가지. df: 입력으로 데이터 프레임을 넣어주고 reader: 이 데이터 프레임이 어떤 포맷으로 되어있습니다 라고 지정 해준다.
 
<img src = "./img/DatasetMethod.png">

<h2>Surprise 주요 모듈 소개 - Reader</h2>

이런 데이터 세트를 데이터 셋으로 로드하기 위해서 미리 사전에 데이터 폼에서 지정을 해준다. 어떻게 하냐면

line_format = "1컬럼 2컬럼 3컬럼 4컬럼 에서 자동으로 첫번재 userId 두번재 ItemId 세번째는 평점으로 자동적으로 인지를 하게 된다.

지금까지했던 무비렌즈는 최소평점이 0.5~5인데 안그럴 수도 있다. 안 그런 경우에는 rating_scale로 지정해주면 된다.

load_from_file로 파일 디렉토리를 지정해주고 reader를 호출하면 여기 있는 파일을 reader 폼에 맞춰서 입력하게 된다.

근데 sep을 보면 디폴트가 탭이다(탭키) 왜 그럴까? 실습할 때 얘기나오는데 과거 모듈 데이터 셋이라 그땐 디폴트가 탭이었다.

<img src = "./img/SurpriseModule.png">

<h1>서프라이즈를 이용해 기본 추천 구현 실습</h1><br><br>

[Supriselib 바로가기](https://supriselib.com)

In [1]:
# 서프라이즈 버전 확인
import surprise 

print(surprise.__version__)

1.1.1


### Surprise 를 이용한 추천 시스템 구축

In [2]:
from surprise import SVD # 추천 알고리즘
from surprise import Dataset #원본 raw 데이터를 서프라이즈에 적합한 데이터 셋으로 만들어주는 것
from surprise import accuracy # 이 모듈안에 rmse가 있으므로 그걸로 평가할 것
from surprise.model_selection import train_test_split # 이걸로 학습, 테스트 데이터 셋 분리할 것

**내장 데이터를 로드하고 학습과 테스트 데이터로 분리**

In [3]:
data = Dataset.load_builtin('ml-100k') 
trainset, testset = train_test_split(data, test_size=.25, random_state=0) 

**추천 행렬 분해 알고리즘으로 SVD객체를 생성하고 학습수행**

In [4]:
algo = SVD()
algo.fit(trainset) 

**테스트 데이터 세트에 예상 평점 데이터 예측. test()메소드 호출 시 Prediction 객체의 리스트로 평점 예측 데이터 반환**

In [5]:
predictions = algo.test(testset)
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.7655400341312286, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.6061742294724124, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.204219862347287, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.6596559285045807, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.1710828572267284, details={'was_impossible': False})]

확인해보니 list 클래스이고 25000개의 원소를 가진다. 5개만 추출해서 어떤 데이터가 들어가있는지 확인해보니까

Prediction 객체가 들어가 있고 uid, iid, r_ui(실제 평점) est(예측평점). 이런 세트가 25000개 리스트에 들어가 있는거다.

r_ui는 필요없으니까 아래처럼 간단히 뽑을 수 있다.

In [6]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.7655400341312286),
 ('882', '291', 3.6061742294724124),
 ('535', '507', 4.204219862347287)]

**predict()메소드는 개별 사용자,아이템에 대한 예측 평점 정보를 반환**

In [7]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함. 
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.03   {'was_impossible': False}


객체 하나만 반환된거다. item 302에 대해서 평가를 하진 않았지만 est(estimate)는 4.08이다 라는 뜻

**반환된 Prediction의 리스트 객체를 기반으로 RMSE 평가**

In [8]:
accuracy.rmse(predictions)

RMSE: 0.9474


0.9474254414864771

### Surprise 주요 모듈 소개
**csv 파일로 사용자 평점 데이터 생성**

In [9]:
import pandas as pd

ratings = pd.read_csv('./ml-latest-small/ratings.csv')
# ratings_noh.csv 파일로 unload 시 index 와 header를 모두 제거한 새로운 파일 생성.  
ratings.to_csv('./ml-latest-small/ratings_noh.csv', index=False, header=False)


**Reader클래스로 파일의 포맷팅 지정하고 Dataset의 load_from_file()을 이용하여 데이터셋 로딩**

csv 파일이라고 알려줘야 하므로 지정한다.

In [10]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('./ml-latest-small/ratings_noh.csv',reader=reader)

**학습과 테스트 데이터 세트로 분할하고 SVD로 학습후 테스트데이터 평점 예측 후 RMSE평가**

이제 앞에하고 똑같다. 똑같은데 다른게 뭐냐면 앞에는 SVD에 디폴트를 했다.

여기선 n_factor = 50개를 줄거다. 이건 앞에서의 K 값임

In [11]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정 
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)


RMSE: 0.8682


0.8681952927143516

**판다스 DataFrame기반에서 동일하게 재 수행**

이것도 똑같다. 대신 load_from_df (df에서 바로 로드하겠다는 함수를 호출)

reader에 별 지정을 안했다. 데이터프레임은 그럴 필요가 없다. 그냥 데이터 프레임만 넣어주면 됨. 그리고 reader는 스케일만 넣어준다.

**단 사용자 아이디, 아이템 아이디, 평점 순서를 반드시 지켜야 한다.**

In [12]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('./ml-latest-small/ratings.csv') 
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다. 
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)


RMSE: 0.8682


0.8681952927143516

<h1>Surprise 추천 알고리즘 클래스</h1>

서프라이즈를 조금 더 파고 들어보자.

대표적인 클래스는

SVD: 행렬 분해

KNNBasic: 사용자 기반이냐 아이템 기반이냐에 따라서 최근접(K-nearest)

BaselineOnly: 사용자 Bias 아이템 Bias를 감안

[지원 알고리즘은 suprise 사이트 문서 참조](http://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html)

<img src="./img/SurpriseAlgoClass.png">

<h2>사용자의 성향을 반영한 Baseline rating</h2>

특정 사용자 u가 아직 평가를 하지 않은 아이템 i 에 대해 어떻게 평점을 내리냐면 왼쪽 수식대로이다.

사용자의 성향을 반영하는거다. 평점을 줄때 깐깐한 사람이라던가. 

<img src="./img/BaselienRating.png">

<h2>Baseline rating을 반영한 행렬 분해의 비용 최소화 함수</h2>

SVD를 SGD를 할 때 사용자 baseline rating을 안했었으나 일반적으로는 감안해서 한다.

b^i+ b^2u를 최소화할 수 있는 p값과 q 값을 구해서 계속 업데이트를 시키는데 Gradient decent 방식

<img src="./img/BaselienRating1.png">

<h2>SVD의 튜닝 파라미터</h2>

3개 밖에 없다.

n_factors: K개 차원. 이게 몇개냐. 이걸 크게할 수록 정확도는 높아질 수 밖에 없다(원래 차원으로 가므로..)그러나 과적합 문제 발생.

n_epochs: SGD 반복 횟수. 디폴트 20

biased (bool): 디폴트는 True인데 베이스라인 편향을 적용할거냐 안할거냐.

<img src="./img/SVDParam.png">

<h2>교차 검증과 하이퍼 파라미터 튜닝</h2>

**Surprise는 교차 검증과 하이퍼 파라미터 튜닝을 위해 사이킷런과 유사한 cross_validate()와 GridSearchCV 클래스를 제공합니다.**

### 교차 검증(Cross Validation)과 하이퍼 파라미터 튜닝

**cross_validate()를 이용한 교차 검증**

In [13]:
from surprise.model_selection import cross_validate 

# Pandas DataFrame에서 Surprise Dataset으로 데이터 로딩 
ratings = pd.read_csv('./ml-latest-small/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0) 
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) 

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8718  0.8808  0.8740  0.8738  0.8641  0.8729  0.0053  
MAE (testset)     0.6703  0.6768  0.6722  0.6709  0.6642  0.6709  0.0040  
Fit time          3.12    3.12    3.12    3.11    3.13    3.12    0.01    
Test time         0.10    0.12    0.12    0.09    0.08    0.10    0.02    


{'test_rmse': array([0.87184373, 0.88075985, 0.87400287, 0.87383202, 0.86408146]),
 'test_mae': array([0.6702703 , 0.67679466, 0.67219773, 0.67091898, 0.66422805]),
 'fit_time': (3.124035358428955,
  3.116971969604492,
  3.123969316482544,
  3.1050002574920654,
  3.126000165939331),
 'test_time': (0.09600687026977539,
  0.12303042411804199,
  0.125,
  0.08597135543823242,
  0.08499884605407715)}

RMSE, MAE에 대해서 개별 폴드셋 별로 개별적인 스코어가 나왔다.

**GridSearchCV 이용**

유사하다. param_grid를 통해서 앞에 세개의 SVD 행렬 분해 방식의 n_epochs, n_factors. bias는 기본적으ㅜ로 가져간다.

그리고 GridSearchCV를 생성하는데 유의해야 할게 있다. SVD라고 그냥 클래스 명이 들어간다. 쉬프트 탭을 눌러서 한번 보자.

이걸 만약에 Algo 이렇게 넣으면 오류가 난다. Algo는 위에 보면 이미 초기화가 들어간다.

그리고 best_score. 약간 사이킷런에서랑 다르다. 

In [14]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])




0.8781543551293467
{'n_epochs': 20, 'n_factors': 50}


### Surprise 를 이용한 개인화 영화 추천 시스템 구축

**SVD 학습은 TrainSet 클래스를 이용해야 함**

먼저 전에 안보던 클래스를 써야한다. DatasetAutoFolds라는 클래스.

이전에는 train_test_splits으로 나눠서 train 세트에 SVD를 fit했었다. 

이렇게 안나누고 그냥하면 받아들이지 못한다. 근데 분리를 안하고 하는 방법이 있다.

그게 바로 DatasetAutoFolds라는 것임.  데이터셋 로딩하는거랑 똑같은데 그 앞에 이것만 들어갔다.

전체 데이터를 학습데이터로 생성할 수 있음. 이게 앞에 했던거랑 살짝 다르다.

In [15]:
# 아래 코드는 train_test_split( )으로 분리되지 않는 Dataset에 fit( )을 호출하여 오류를 발생합니다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)


AttributeError: 'DatasetAutoFolds' object has no attribute 'global_mean'

**DatasetAutoFolds를 이용한 전체 데이터를 TrainSet클래스 변환**

In [16]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='./ml-latest-small/ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()


**SVD로 학습**

In [17]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)


학습이 되었다.

이제 rating이 아니라 movies.csv로 영화에 대한 타이틀 정보를 필터링하기 위해서 데이터프레임으로 로딩할거다.

그리고 9번 유저. 이 9번 유저에 대해서 특정 영화들을 추천할거다.

영화 42의 평점을 어떻게 먹이는지 한번 보자.(아직 9번 유저가 평가하지 않았음)

In [18]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('./ml-latest-small/movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])


사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [19]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)


user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


위 내용: 9 번 사용자에 대해서 42번 아이템에 대해서 예측 평점을 추측해주세요

In [20]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


이번에는 surprise를 통해서 추천을 할거다.

prediction을 할건데 개별영화를 반복적으로 호출하면서 할거다. 

무슨 얘끼냐면 unseen_movies에 평점을 46개 먹였으니까 전체 9742개중에 9696개는 추천영화 중에 들어갈 수 있다.

리스트 컴프리헨션으로 루프를 돌면서 algo.predict를 호출하세요 라는 코드(predictions)로.

In [21]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
